In [1]:
import phoebe as pb
from phoebe import u
import numpy as np
import matplotlib.pyplot as plt

This next cell has to be run in order to get around a bug that is in the PHOEBE code itself (see the latest Git issue tag for explanation).

In [3]:
pb.atmospheres.passbands._url_tables_server = 'https://staging.phoebe-project.org'
pb.list_online_passbands(refresh=True)
pb.list_all_update_passbands_available()
pb.update_all_passbands()


This initiates a default binary system that you can change the parameter values of. The dataset is an empty dataset that generates a list of times and any changes to parameters will be added to this dataset (I believe).

In [4]:
logger = pb.logger()
b = pb.default_binary()
b.add_dataset('lc', times=pb.linspace(0,1,101), dataset='lc01', overwrite=True)

<ParameterSet: 80 parameters | contexts: constraint, dataset, compute, figure>

There's a couple different ways that you can set parameter values. Below is how I set them.
There's a lot of parameters that are set to default values, so you don't need to set them all.

If you want to see what the default values are, you can use the command `print(b.filter())` to see all the parameters that are set to default values.
Changing the value of a parameter is as simple as `b['parameter_name'] = value`. For example, `b['q'] = 0.5` would set the mass ratio to 0.5.
If you want to see what parameters are available for a given component, you can use the command `print(b['component_name'].filter())`. For example, `print(b['primary'].filter())` would show you all the parameters that are available for the primary component.
If you want to see what parameters are available for a given dataset, you can use the command `print(b['dataset_name'].filter())`. For example, `print(b['lc01'].filter())` would show you all the parameters that are available for the lc01 dataset.
If you want to see what parameters are available for a given constraint, you can use the command `print(b['constraint_name'].filter())`. For example, `print(b['requiv@primary'].filter())` would show you all the parameters that are available for the requiv@primary constraint.

You cannot set some parameters immediately without addressing hierarchy. For example, you cannot set `b['mass'] = 1.0` because mass is a parameter of the primary and secondary components. You would need to set `b['mass@primary'] = 1.0` instead.
Sometimes you can't set a parameter without 'flipping' the hierarchy. You'll know you're running into this problem if you get the error "ValueError: cannot change the value of a constrained parameter." If you run into issues, you can look at this tutorial:
https://phoebe-project.org/docs/2.4/tutorials/constraints_hierarchies

I left in some other lines of how to flip and set some other parameters. If you uncomment them here there's no promise they work in this order but you can at least see how to set them

In [5]:

#binary parameters
b["ecc@binary"] = 0.26
b["q@binary"] = 1
# b["incl@binary"] = 79.5
# b["per0"] = 275 *u.deg
# b['teffratio@binary'] = 0.21


#primary parameters
b.flip_constraint("sma@binary", solve_for="mass")
b["mass@primary@component"] = 13.5
b["requiv@primary"] = 1 
b["teff@primary"] = 7500
# b.flip_constraint("logg@primary", solve_for="requiv@primary")
# b.flip_constraint("requiv@primary",solve_for="logg@primary")

#secondary parameters
# b.flip_constraint("mass@secondary", solve_for="q")
# b["mass@secondary@component"] = 1.8

print(b.filter())

ParameterSet: 187 parameters
                        t0@system: 0.0 d
                        ra@system: 0.0 deg
                       dec@system: 0.0 deg
                  distance@system: 1.0 m
                    vgamma@system: 0.0 km / s
C                      ebv@system: 0.0
                        Av@system: 0.0
                        Rv@system: 3.1
                 hierarchy@system: orbit:binary(star:primary, star:secondary)
         requiv@primary@component: 1.0 solRad
C    requiv_max@primary@component: 1.5462069723476948 solRad
           teff@primary@component: 7500.0 K
           abun@primary@component: 0.0
C          logg@primary@component: 4.437551868254479
        syncpar@primary@component: 1.0
C        period@primary@component: 1.0 d
C          freq@primary@component: 6.283185307179586 rad / d
          pitch@primary@component: 0.0 deg
            yaw@primary@component: 0.0 deg
C          incl@primary@component: 90.0 deg
C       long_an@primary@component: 0.0 deg
     

This next cell actually computes the model:

In [6]:
b.run_compute(irrad_method='none')

100%|██████████| 101/101 [00:01<00:00, 82.00it/s]


<ParameterSet: 3 parameters | qualifiers: fluxes, times, comments>

This next cell allows you to view the lightcurve in the figure 'pbtest.png' which will be in the same directory as where you run this file. There's some matplotlib issue that I can't figure out how to solve right now that doesn't allow the figure to be shown in this notebook.

In [7]:
plotter = b.plot(x='phase',  ylabel = 'Normalized Flux', xlabel = 'Phase', show=True, save = 'pbtest.png')
# plt.ion()
plt.show(block=False)

/opt/anaconda3/envs/phoebe_environment/lib/python3.13/site-packages/phoebe/dependencies/autofig/figure.py:627: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()  # <-- blocking
/var/folders/34/3847lqd14j78mfm70_z1c_r00000gn/T/ipykernel_19762/3563954529.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show(block=False)


You can then save an output file (format is by default JSON) of the binary:

In [ ]:
print(b.save('test.phoebe'))